In [9]:
from langchain_core.runnables import RunnableConfig

from retrieval_agents import adaptive_rag_graph, configuration

In [10]:
config = RunnableConfig(configurable={"user_id": "test_user"})

In [11]:
graph = adaptive_rag_graph.graph

In [13]:
result = await graph.ainvoke({"question": "test"}, config=config)

DEBUG:retrieval_graph.adaptive_rag_graph:LLM: openai/gpt-4o
DEBUG:retrieval_graph.adaptive_rag_graph:structured_llm created


TypeError: Object of type 'FieldInfo' is not JSON serializable

In [17]:
result = await adaptive_rag_graph.route_question({"question": "test"}, config=config)

DEBUG:retrieval_graph.adaptive_rag_graph:LLM: openai/gpt-4o


DEBUG:retrieval_graph.adaptive_rag_graph:structured_llm created


AttributeError: 'dict' object has no attribute 'question'

In [1]:
### Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
# print(
#    question_router.invoke(
#        {"question": "Who will the Bears draft first in the NFL draft?"}
#    )
# )
# print(question_router.invoke({"question": "What are the types of agent memory?"}))

r = await question_router.ainvoke({"question": "What are the types of agent memory?"})